In [1]:
#from _future_ import print_function, division
from __future__ import print_function, division
import time
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['figure.figsize'] = (13,6)
from nilmtk import DataSet,MeterGroup
import warnings
warnings.filterwarnings("ignore")

In [2]:
train = DataSet("/Volumes/MacintoshHD2/Users/haroonr/Downloads/iawe.h5")
test  = DataSet("/Volumes/MacintoshHD2/Users/haroonr/Downloads/iawe.h5")

In [3]:
#building = 1
#train.buildings[building].elec.mains().plot()

In [4]:
train.set_window(end="2013-07-01")
test.set_window(start ="2013-07-01")
train_elec = train.buildings[1].elec
test_elec = test.buildings[1].elec

In [5]:
#train_elec.mains().plot()

In [6]:
#test_elec.mains().plot()

In [7]:
#train.buildings[1].elec

In [8]:
# concentrate on one applaince, and see its sampling rate
fridge_meter = train_elec["fridge"]
fridge_df = fridge_meter.load().next()
fridge_df.head(2)

physical_quantity,voltage,power,frequency,power,current,power
type,,apparent,,active,,reactive
2013-06-07 05:30:00+05:30,235.070007,2.486,50.070000,0.111,0.011,2.483
2013-06-07 05:30:01+05:30,235.020004,2.555,50.080002,0.200,0.011,2.547


In [9]:
mains = train_elec.mains()
mains_df = mains.load().next()
mains_df.head(2)

Loading data for meter ElecMeterID(instance=2, building=1, dataset='iAWE')     
Done loading data all meters for this chunk.


physical_quantity,power,power factor,power,voltage,current,power,frequency
type,apparent,NaN,active,apparent,NaN,reactive,apparent
2013-05-24 05:30:00+05:30,295.160004,NaN,282.744995,NaN,NaN,-52.148102,NaN
2013-05-24 05:30:01+05:30,307.010010,NaN,288.757996,NaN,NaN,-59.433601,NaN


In [10]:
# selecting top 5 applainces based on energy consumption
top_5_train_elec =  train_elec.submeters().select_top_k(k=5)
#top_5_train_elec

10/10 ElecMeter(instance=12, building=1, dataset='iAWE', appliances=[Appliance(type='motor', instance=1)])

In [11]:
# FHMM DISSAGREGATION
start = time.time()
from nilmtk.disaggregate import fhmm_exact
fhmm = fhmm_exact.FHMM()
fhmm.train(top_5_train_elec,sample_period = 60)
end = time.time()
print("runtime=",end - start , "seconds")

Training model for submeter 'ElecMeter(instance=7, building=1, dataset='iAWE', appliances=[Appliance(type='computer', instance=1)])'
Training model for submeter 'ElecMeter(instance=5, building=1, dataset='iAWE', appliances=[Appliance(type='air conditioner', instance=2)])'
Training model for submeter 'ElecMeter(instance=4, building=1, dataset='iAWE', appliances=[Appliance(type='air conditioner', instance=1)])'
Training model for submeter 'ElecMeter(instance=3, building=1, dataset='iAWE', appliances=[Appliance(type='fridge', instance=1)])'
Training model for submeter 'ElecMeter(instance=10, building=1, dataset='iAWE', appliances=[Appliance(type='television', instance=1)])'
runtime= 13.8668880463 seconds


In [12]:
top_5_train_elec.info()

AttributeError: 'MeterGroup' object has no attribute 'info'

In [13]:
#test_elec.mains().load(physical_quantity = 'power',ac_type='reactive').next()

In [14]:
from nilmtk import HDFDataStore
test_dat = mains_df['power']['apparent']
#test_elec.mains(),
disagg_filename = "/Volumes/MacintoshHD2/Users/haroonr/Dropbox/nilmtk_work/iawe_disag_fhmm.h5"
output =  HDFDataStore(disagg_filename,'w')
#fhmm.disaggregate(test_dat,output,sample_period=60)
#test_elec.mains().load(physical_quantity = 'power',ac_type='reactive').next()
#fhmm.disaggregate(test_elec.mains().load().next()['power']['apparent'],output,sample_period=60)
fhmm.disaggregate(test_elec.mains().load(physical_quantity = 'power',ac_type='reactive').next(),output,sample_period=60)
ouput.close()

Loading data for meter ElecMeterID(instance=2, building=1, dataset='iAWE')     
Done loading data all meters for this chunk.


AttributeError: 'DataFrame' object has no attribute 'good_sections'

In [15]:
#output.close()
#test_elec.mains().load().next()['power']['apparent']
test_elec.mains().load(physical_quantity = 'power',ac_type='reactive').next().head()

Loading data for meter ElecMeterID(instance=2, building=1, dataset='iAWE')     
Done loading data all meters for this chunk.


physical_quantity,power
type,reactive
2013-07-01 00:00:00+05:30,77.132896
2013-07-01 00:00:01+05:30,77.132896
2013-07-01 00:00:02+05:30,77.132896
2013-07-01 00:00:03+05:30,81.619797
2013-07-01 00:00:04+05:30,81.619797
